# Problema 2: Implementación del Método Simplex Dual Phase

In [39]:
import numpy as np
import pandas as pd
def show_table(table: pd.DataFrame, title: str = ""):
    from tabulate import tabulate
    
    if title:
        print(f"=== {title} ===")
    print(tabulate(table, headers="keys", tablefmt="psql", showindex=True))
    print()  # Separación final



### 0. Problema a minimizar

<p align="center">
Minimizar <br>
Z = 5x1 − 4x2 + 3x3<br>
sujeto a<br>
2x1 + x2 − x3 = 10<br>
x1 − 3x2 + 2x3 ≥ 5<br>
x1 + x2 + x3 ≤ 15<br>
x1, x2, x3 ≥ 0
</p>

# FASE 1

#### a. Introducir las variables necesarias
#### b. Formular el problema auxiliar que minimiza la suma de variables artificial

<p align="center">
Maximizar la f. o. auxiliar (min w = x4 + x6) <br>
max w' = -x4 -x6 <br>
sujeto a<br>
2x1 + x2 − x3 + x4 = 10<br> 
x1 − 3x2 + 2x3 -x5 + x6 = 5<br>
x1 + x2 + x3 +x7 = 15<br>
</p>

In [40]:


# x1, x2, x3: variables originales del problema
# x4: variable artificial para la restricción 1 (igualdad)
# x5: variable de exceso para la restricción 2 (≥)
# x6: variable artificial para la restricción 2 (≥)
# x7: variable de holgura para la restricción 3 (≤)
# ld: lado derecho de cada ecuación 

                        # x1, x2, x3, x4, x5, x6, x7, ld
restricciones = np.array([
                         [ 2, 1 , -1, 1 , 0 , 0 , 0, 10 ], # R1: Igualdad -> se añade x4 (artificial)
                         [ 1, -3, 2, 0 , -1 , 1 , 0, 5 ], # R2: ≥ -> se resta x5 (exceso) y se añade x6 (artificial)
                         [ 1, 1 , 1, 0 , 0 , 0 , 1, 15 ]  # R3: ≤ -> se añade x7 (holgura)
                         ])
columnas = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'lado derecho']
base = ['x4', 'x6', 'x7']  # variables básicas 
artificiales = ['x4', 'x6']  # variables artificiales
coeficientes_z = { # coeficientes de la función objetivo Z multiplicados por -1, pues se quiere minimizar
    'x1': -5,
    'x2': 4,
    'x3': -3,
    'x5': 0,
    'x7': 0,
    'lado derecho': 0
}
tabla_inicial = pd.DataFrame(restricciones, columns=columnas, index=base)


tabla_inicial.loc['Z'] = [ 0, 0, 0, -1, 0, -1, 0, 0]  # agregar fila Z con el problema auxiliar 
print("Tabla inicial (Fase I - antes de aplicar Simplex):")
show_table(tabla_inicial)


Tabla inicial (Fase I - antes de aplicar Simplex):
+----+------+------+------+------+------+------+------+----------------+
|    |   x1 |   x2 |   x3 |   x4 |   x5 |   x6 |   x7 |   lado derecho |
|----+------+------+------+------+------+------+------+----------------|
| x4 |    2 |    1 |   -1 |    1 |    0 |    0 |    0 |             10 |
| x6 |    1 |   -3 |    2 |    0 |   -1 |    1 |    0 |              5 |
| x7 |    1 |    1 |    1 |    0 |    0 |    0 |    1 |             15 |
| Z  |    0 |    0 |    0 |   -1 |    0 |   -1 |    0 |              0 |
+----+------+------+------+------+------+------+------+----------------+



#### c. Resolver este problema auxiliar utilizando el método Simplex estándar

siguiendo los pasos del laboratorio: 

 1. VERIFICAR OPTIMALIDAD
 -  Calcular los costos reducidos ¯cj para cada variable no básica.

In [41]:
def calcular_costos_reducidos(tabla, base):
    #sacar z de la tabla
    z = np.zeros(tabla.shape[1]) # inicializar la fila Z con ceros

    # multiplcar la fila de cada base por su coeficiente en la fila Z ej: x4 -> [2,1,-1,1,0,0,0,10] * 0
    for b in base:
        coef = tabla.loc['Z', b]
        fila = tabla.loc[b]
        z += coef * fila
    
    tabla.loc['Z'] = z  # actualizar fila Z con los costos reducidos
    return tabla

tabla_actualizada = calcular_costos_reducidos(tabla_inicial.copy(), base)
print("\nTabla inicial:")
show_table(tabla_actualizada)



Tabla inicial:
+----+------+------+------+------+------+------+------+----------------+
|    |   x1 |   x2 |   x3 |   x4 |   x5 |   x6 |   x7 |   lado derecho |
|----+------+------+------+------+------+------+------+----------------|
| x4 |    2 |    1 |   -1 |    1 |    0 |    0 |    0 |             10 |
| x6 |    1 |   -3 |    2 |    0 |   -1 |    1 |    0 |              5 |
| x7 |    1 |    1 |    1 |    0 |    0 |    0 |    1 |             15 |
| Z  |   -3 |    2 |   -1 |   -1 |    1 |   -1 |    0 |            -15 |
+----+------+------+------+------+------+------+------+----------------+



- Si todos los costos reducidos son no negativos (para maximización), la solución actual es óptima. Terminar.

In [42]:
def verificar_optimalidad(tabla):
    # Verificar si todos los costos reducidos son no negativos
    return all(tabla.loc['Z'][:-1] >= 0)  # Exepto el lado derecho

2. Seleccionar variable de entrada

- Elegir una variable no básica con costo reducido negativo (para maximización) para ingresar a la base.
- Regla de Dantzig: seleccionar la variable con el costo reducido más negativo.

In [43]:
def seleccionar_var_entrada(tabla, base):
    z_values = tabla.loc['Z'][:-1]  # Excluyendo el lado derecho
    z_values = z_values.drop(labels=base, errors='ignore')  # ← evitar repetir variables de la base
    indice_mas_neg = np.argmin(z_values)  # Posicion del costo reducido más negativo
    valor = z_values.iloc[indice_mas_neg]  # Valor del costo reducido más negativo

    if valor >= 0:
        return None
    var_entrada = z_values.index[indice_mas_neg]
    print(f"Variable de entrada: {var_entrada} (costo reducido = {valor:.4f})")
    return var_entrada


3. Calcular razones
- Para cada restricción i donde el coeficiente aij de la variable de entrada j es positivo, calcular la razón θi = bi / aij.
- Si todos los coeficientes aij ≤ 0, el problema es ilimitado. Terminar.
4. Seleccionar variable de salida
- La variable básica asociada a la restricción con la menor razón θi sale de la base (regla del mínimo cociente).
- Este paso determina el elemento pivote ars en la fila r y columna s.

In [44]:
def calcular_razones(tabla, var_entrada):
    # Calcular las razones para determinar la variable de salida
    razones = []
    filas_basicas = tabla.index[:-1]  # Excluyendo la fila Z
    for fila in filas_basicas:
        coef = tabla.loc[fila, var_entrada]
        ld = tabla.loc[fila, 'lado derecho']
        if coef > 0:
            razon = ld / coef
            razones.append(razon)
        else:
            razones.append( np.inf)
    idx_var_salida = np.argmin(razones)
    var_salida = filas_basicas[idx_var_salida]
    print(f"Variable de salida: {var_salida} (razón mínima = {razones[idx_var_salida]:.4f})")
    return var_salida

5.  Actualizar la tabla (operación de pivoteo):
- Dividir la fila del pivote r por el valor del pivote ars.
- Para cada fila i ̸= r: Restar de la fila i la fila pivote multiplicada por ais.

-> Dividir fila pivote </br>
-> Resto de filas: fila = fila - coef * fila pivote

- Actualizar la lista de variables básicas (reemplazar la variable que sale por la que entra).

In [45]:
def pivoteo(tabla, var_entrada, var_salida):
    nueva_tabla = tabla.copy()
    print(f"\nPivoteo: {var_entrada} entra, {var_salida} sale")
    pivote = nueva_tabla.loc[var_salida, var_entrada]  # Elemento pivote
    print(f"Elemento pivote: {pivote}")
    nueva_tabla.loc[var_salida] = nueva_tabla.loc[var_salida] / pivote  

    # volver 0 los demás elementos de la columna de la variable de entrada
    for fila in nueva_tabla.index:
        if fila != var_salida:
            coef = nueva_tabla.loc[fila, var_entrada]
            nueva_tabla.loc[fila] = nueva_tabla.loc[fila] - coef * nueva_tabla.loc[var_salida]
    
    # Actualizar la variable básica
    
    nueva_base =  []
    for fila in nueva_tabla.index[:-1]:
        if fila == var_salida:
            nueva_base.append(var_entrada)  # Cambiar la variable básica , reemplazar la variable que sale por la que entra
        else:
            nueva_base.append(fila) # Mantener la variable básica
    nueva_tabla.index = nueva_base + ['Z']  # Actualizar el índice de la tabla
    return nueva_tabla, nueva_base  

6. Simplex e iterar volviendo al paso 2

In [46]:
def simplex(tabla, base):
    iteracion = 0

    while True:
        print(f"\n Iteración {iteracion}")
        tabla = calcular_costos_reducidos(tabla, base)  # Calcular costos reducidos

        print("Costos reducidos:")
        show_table(tabla) 

        if verificar_optimalidad(tabla):
            print("\n Solución óptima alcanzada (Fase 1 terminada).")
            break

        var_entrada = seleccionar_var_entrada(tabla, base)  # Seleccionar variable de entrada

        if var_entrada is None:
            print("No hay variable de entrada. Ha terminado el algoritmo.")
            break

        var_salida = calcular_razones(tabla, var_entrada)

        tabla, base = pivoteo(tabla, var_entrada, var_salida)  # Pivoteo actualiza tabla y base

        print(" Tabla después del pivoteo:")
        show_table(tabla) 
        iteracion += 1

    return tabla, base

tabla_inicial= tabla_inicial.astype(float)  
tabla_final, base_final = simplex(tabla_inicial.copy(), base.copy())




 Iteración 0
Costos reducidos:
+----+------+------+------+------+------+------+------+----------------+
|    |   x1 |   x2 |   x3 |   x4 |   x5 |   x6 |   x7 |   lado derecho |
|----+------+------+------+------+------+------+------+----------------|
| x4 |    2 |    1 |   -1 |    1 |    0 |    0 |    0 |             10 |
| x6 |    1 |   -3 |    2 |    0 |   -1 |    1 |    0 |              5 |
| x7 |    1 |    1 |    1 |    0 |    0 |    0 |    1 |             15 |
| Z  |   -3 |    2 |   -1 |   -1 |    1 |   -1 |    0 |            -15 |
+----+------+------+------+------+------+------+------+----------------+

Variable de entrada: x1 (costo reducido = -3.0000)
Variable de salida: x4 (razón mínima = 5.0000)

Pivoteo: x1 entra, x4 sale
Elemento pivote: 2.0
 Tabla después del pivoteo:
+----+------+------+------+------+------+------+------+----------------+
|    |   x1 |   x2 |   x3 |   x4 |   x5 |   x6 |   x7 |   lado derecho |
|----+------+------+------+------+------+------+------+-------

#### d.  Verificar si se encontró una solución básica factible para el problema original.

Si es una SBF, pues todos los valores de Z son mayores a 0

Ya que se encontró la SBF, se prosigue con la fase 2

## FASE 2

1. Preparar la tabla Simplex:
-  Eliminar todas las variables artificiales.



In [47]:
def eliminar_artificiales(tabla,base, artificiales):
    tabla_final = tabla.copy()
    # Eliminar variables artificiales de la tabla
    tabla_final = tabla_final.drop(artificiales, axis=1)
    base_final = [var for var in base if var not in artificiales]  # Actualizar la base eliminando variables artificiales

    return tabla_final, base_final


-  Restablecer la función objetivo original.

In [48]:
def restablecer_f_o(tabla, coeficientes_z):
    tabla.loc['Z'] = [coeficientes_z.get(col, 0) for col in tabla.columns]
    return tabla


-  Calcular la fila de costos reducidos para la función objetivo original.

2. Aplicar el método Simplex estándar:
-  Continuar con el método Simplex estándar desde la SBF encontrada en la Fase I.
- Retornar la solución óptima o indicar que el problema es ilimitado.

In [49]:
def simplex_fase_2(tabla, base):

    tabla_sin_art, base_sin_art = eliminar_artificiales(tabla, base, artificiales)
    tabla_f_o_rest = restablecer_f_o(tabla_sin_art, coeficientes_z)
    tabla_fase_2, base_fase_2 = simplex(tabla_f_o_rest.copy(), base_sin_art.copy())
    return tabla_fase_2, base_fase_2  # Devolver la tabla y la base de la fase 2

tabla_fase_2, base_fase_2 = simplex_fase_2(tabla_final.copy(), base_final.copy())
print("\nTabla final (Fase II):")
show_table(tabla_fase_2)  # Mostrar la tabla final de la fase II


## PARA IMPRIMIR
variables_originales = ['x1', 'x2', 'x3']

print("\n Solución óptima (x1,x2,x3 y Z):")
for var in variables_originales:
    if var in base_fase_2:
        valor = tabla_fase_2.loc[var, 'lado derecho']
    else:
        valor = 0.0
    # Mostrar el valor de cada variable
    print(f"{var} = {valor.__float__():.1f}")

valor_otimo = -tabla_fase_2.loc['Z', 'lado derecho']  # Valor óptimo de Z, se pone el signo negativo porque es un problema de minimización
print(f"\n Valor óptimo de Z: {valor_otimo}")


 Iteración 0
Costos reducidos:
+----+------+------+------+------+------+----------------+
|    |   x1 |   x2 |   x3 |   x5 |   x7 |   lado derecho |
|----+------+------+------+------+------+----------------|
| x1 |    1 | -0.2 |    0 | -0.2 |    0 |              5 |
| x3 |    0 | -1.4 |    1 | -0.4 |    0 |              0 |
| x7 |    0 |  2.6 |    0 |  0.6 |    1 |             10 |
| Z  |   -5 |  5.2 |   -3 |  2.2 |    0 |            -25 |
+----+------+------+------+------+------+----------------+

No hay variable de entrada. Ha terminado el algoritmo.

Tabla final (Fase II):
+----+------+------+------+------+------+----------------+
|    |   x1 |   x2 |   x3 |   x5 |   x7 |   lado derecho |
|----+------+------+------+------+------+----------------|
| x1 |    1 | -0.2 |    0 | -0.2 |    0 |              5 |
| x3 |    0 | -1.4 |    1 | -0.4 |    0 |              0 |
| x7 |    0 |  2.6 |    0 |  0.6 |    1 |             10 |
| Z  |   -5 |  5.2 |   -3 |  2.2 |    0 |            -25 |
+--

#### Analisis:
 ##### Se encontraron diferentes conclusiones luego de utilizar la anterior implementacion de simplex dual phase. En primer lugar, La solucion obtenida luego de aplicar simplex fase 1 y fase 2 satisface todas las restricciones del problema. Además. la variable x2 no está en las basicas de la solución final, con valor 0, lo que significa que no participa en la combinación óptima. También es importante mencionar que la solucion optima ya habia sido encontrada en la Fase 1, lo que se evidencia en la fase 2 pues no surgen más iteraciones, luego de calcular los costos reducidos de la tabla de entrada a simplex fase 2 obtenemos la solucion optima para los valores de la ecuacion y el valor optimo de Z.

